## Exercises Part 4

Упражнения по 4 главе 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Exercises Part 4")
hdfs = "hdfs://localhost:9000/"

#### Exe. 1

Вывести k наибольших значений.

<h5>
Mapper:
</h5>

In [ ]:
# attribute_k_max_mapper.py 
import sys
import operator

index = int(sys.argv[1]) if len(sys.argv) > 1 else 8
k = int(sys.argv[2]) if len(sys.argv) > 2 else 5

class Mapper:
    __max_values_list = []
    __last_max_value = 0
    
    def __init__(self, k, index):
        self.k = k
        self.index = index
    
    def map(self, key, value):
        fields = value.split(",")
        if len(fields) > 8:
            if fields[self.index].isdigit():
                val = int(fields[self.index])
                if val > self.__last_max_value:
                    self.__last_max_value = val
                    self.__max_values_list.append(fields)
                    self.__max_values_list = sorted(self.__max_values_list, key=operator.itemgetter(self.index), reverse=True)[:self.k]

    def close(self):
        for val in self.__max_values_list:
            if len(val):
                print(','.join(val))

mapper = Mapper(k, index)
for i, line in enumerate(sys.stdin):
    if line.replace(" ", ""):
        mapper.map(i, line)
mapper.close()


In [ ]:
hadoop jar $HADOOP_HOME/libexec/share/hadoop/tools/lib/hadoop-streaming-2.8.0.jar \ 
    -D mapreduce.job.reduces=1 \ 
    -input /user/askar/apat63_99.txt \
    -output /user/askar/output \ 
    -mapper "python3 /Users/shabykov/Hadoop/attribute_k_max_mapper.py 8 4"
    -reducer "python3 /Users/shabykov/Hadoop/attribute_k_max_mapper.py 8 4"

In [61]:
out_data = sc.textFile(hdfs + "/user/askar/output/part-00000")

for val in out_data.collect():
    print(val.split(","))

['5095054', '1992', '11757', '1990', '"DE"', '""', '618615', '2', '868', '524', '1', '15', '40', '111', '0.975', '0.8442', '0.8586', '4.1622', '14.45', '0.0571', '0.05', '0.0093', '0.009\t']
['\t']
['3862265', '1975', '5499', '1972', '"US"', '"TX"', '186590', '2', '80', '525', '1', '15', '7', '106', '1', '0.7517', '0.6122', '15.8491', '5.8571', '0', '0', '0.068', '0.066\t']
['\t']
['3860168', '1975', '5492', '1971', '"US"', '"OH"', '270680', '2', '72', '702', '2', '22', '8', '4', '1', '0.75', '0.6875', '12', '4.875', '0.5714', '0.5', '0', '0\t']
['\t']
['4085139', '1978', '6682', '1976', '"US"', '"MI"', '600480', '2', '706', '564', '1', '14', '7', '1', '0.8571', '0', '0.8333', '14', '9.5714', '0.6', '0.4286', '0', '0\t']
['\t']


#### Exe. 2

